In [1]:
from fastai import *
from fastai.vision import *


# https://towardsdatascience.com/autoencoders-in-the-fastai-library-fa288e1f899a# 

In [2]:
size = 32
batchsize = 32
tfms = get_transforms(do_flip = True)
src = (ImageImageList.from_folder(image_path).label_from_func(lambda x: x))
data = (src.transform(tfms, size=size, tfm_y=True)
 .databunch(bs=batchsize)
 .normalize(imagenet_stats, do_y = False))

NameError: name 'image_path' is not defined